# Fix Environment

In [2]:
!nvidia-smi

Fri Jan 31 21:18:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!rm -rf ./sample_data

# Preprocess Data

In [3]:
import os
import tarfile
import urllib.request

# URLs for the zip files
links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	  'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
  	'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
  	'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
    'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
    'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
    'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
    'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]

IMAGE_DIR = "./images"
DATASET_DIR = "./dataset"
os.makedirs(DATASET_DIR, exist_ok=True)

for index, link in enumerate(links):
    tar = "images_%02d.tar.gz" % (index + 1)
    tar_path = os.path.join(DATASET_DIR, tar)
    urllib.request.urlretrieve(link, tar_path)
    print(f"Downloaded {tar}")

    with tarfile.open(tar_path, "r") as tar:
        for member in tar.getmembers():
            if member.isfile() and member.name.lower().endswith(".png"):
                member.name = os.path.basename(member.name)  # remove the path
                tar.extract(member, path=IMAGE_DIR)
    print(f"Extracted {tar}")

    os.remove(tar_path)

print("Download and extraction complete")

Downloaded images_01.tar.gz
Extracted <tarfile.TarFile object at 0x79093fef0c90>
Downloaded images_02.tar.gz
Extracted <tarfile.TarFile object at 0x79093fd24510>
Downloaded images_03.tar.gz
Extracted <tarfile.TarFile object at 0x79093fee61d0>
Downloaded images_04.tar.gz
Extracted <tarfile.TarFile object at 0x79093e6fc550>
Downloaded images_05.tar.gz
Extracted <tarfile.TarFile object at 0x79093fd86850>
Downloaded images_06.tar.gz
Extracted <tarfile.TarFile object at 0x79093ff5e610>
Downloaded images_07.tar.gz
Extracted <tarfile.TarFile object at 0x79093fe06710>
Downloaded images_08.tar.gz
Extracted <tarfile.TarFile object at 0x79093f4d15d0>
Downloaded images_09.tar.gz
Extracted <tarfile.TarFile object at 0x79094c753e90>
Downloaded images_10.tar.gz
Extracted <tarfile.TarFile object at 0x79093fe493d0>
Downloaded images_11.tar.gz
Extracted <tarfile.TarFile object at 0x79093fd03850>
Downloaded images_12.tar.gz
Extracted <tarfile.TarFile object at 0x79093f040610>
Download and extraction comp

In [7]:
# Create binary matrix CSV that maps all images with its labels
import pandas as pd

ORIGINAL_CSV = "./dataset/Data_Entry_2017_v2020.csv"
BINARY_MATRIX_CSV = "all_labels.csv"

df = pd.read_csv(ORIGINAL_CSV, usecols=["Image Index", "Finding Labels"])
df["Finding Labels"] = df["Finding Labels"].fillna("")

# Get all possible labels
unique_labels_set = set()
labels_series = df["Finding Labels"].str.split("|")
for labels in labels_series:
    for label in labels:
        if label and label != "No Finding":
            unique_labels_set.add(label)
unique_labels = sorted(unique_labels_set)

# Fill in binary matrix CSV
for label in unique_labels:
    df[label] = df["Finding Labels"].apply(lambda x: 1 if label in x.split("|") else 0)
binary_df = df[["Image Index"] + unique_labels]

binary_df.to_csv(BINARY_MATRIX_CSV, index=False)

In [8]:
# Split training and validation data
TRAIN_CSV = "train_labels.csv"
VAL_CSV = "val_labels.csv"

df = pd.read_csv(BINARY_MATRIX_CSV)

RANDOM_SEED = 42  # Chosen arbitrarily for reproducibility
train_df = df.sample(frac=0.8, random_state=RANDOM_SEED)
val_df = df.drop(train_df.index)

train_df.to_csv(TRAIN_CSV, index=False)
val_df.to_csv(VAL_CSV, index=False)

In [9]:
# Move the extracted images files in ./images into their respective folder
import shutil

TRAIN_DIR = "./train"
VAL_DIR = "./val"

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

for image in train_df["Image Index"]:
    src = os.path.join(IMAGE_DIR, image)
    dst = os.path.join(TRAIN_DIR, image)
    if os.path.exists(src):
        shutil.move(src, dst)

for image in val_df["Image Index"]:
    src = os.path.join(IMAGE_DIR, image)
    dst = os.path.join(VAL_DIR, image)
    if os.path.exists(src):
        shutil.move(src, dst)

# Train

In [10]:
import torch
import torch.nn as nn
from torchvision import transforms, models, datasets

In [11]:
num_epochs = 12
batch_size = 32
learning_rate = 1e-4
device = torch.device("cuda") # if torch.cuda.is_available() else "cpu")

In [12]:
import pandas as pd
import PIL

class ChestXrayDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, csv_file, transform=None):
        self.images_dir = images_dir
        self.annotations = pd.read_csv(csv_file)
        self.transform = transform
        self.image_names = self.annotations.iloc[:, 0].values
        self.labels = self.annotations.iloc[:, 1:].values.astype(float)
        self.num_classes = self.labels.shape[1]

        self.class_labels = list(self.annotations.columns)[1:]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.image_names[idx])
        image = PIL.Image.open(img_name).convert('RGB')
        labels = torch.FloatTensor(self.labels[idx])
        if self.transform:
            image = self.transform(image)
        return image, labels

In [13]:
# Data transformations
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [14]:
# Create Datasets and Dataloaders
train_dataset = ChestXrayDataset(
    images_dir=TRAIN_DIR,
    csv_file=TRAIN_CSV,
    transform=transform
)
val_dataset = ChestXrayDataset(
    images_dir=VAL_DIR,
    csv_file=VAL_CSV,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

In [15]:
# Model
num_labels = train_dataset.num_classes
class_labels = train_loader.dataset.class_labels
print(f"Number of labels: {num_labels}")
print(f"Labels: {class_labels}")

model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 14)
)

# Unfreeze fc layers
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

Number of labels: 14
Labels: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 64.6MB/s]


In [16]:
# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
patience = 3
counter = 0
best_val_loss = float("inf")

import time

start_time = time.time()

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)

    val_loss = val_running_loss / len(val_loader.dataset)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {epoch_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")

end_time = time.time()
print("Training complete")
print(f"Training time took: {end_time - start_time:.4f} seconds")

In [ ]:
# Save the model
import datetime

timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
model_filename = f"{timestamp}_resnet50model.pth"

torch.save(model.state_dict(), model_filename)
print("Saved model")

Saved model


In [ ]:
def load_model(model_path, num_labels, device):
    model = models.resnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_labels)
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model = model.to(device)
    return model

def predict(model, dataloader, device, class_labels, threshold=0.3):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = torch.sigmoid(outputs) > threshold
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Print the first 5 predictions with class labels
    for i in range(5):
        predicted_labels = [class_labels[j] for j in range(num_labels) if all_preds[i][j]]
        true_labels = [class_labels[j] for j in range(num_labels) if all_labels[i][j]]
        print(f"Image {i+1}:")
        print(f"  Predicted Labels: {predicted_labels}")
        print(f"  True Labels: {true_labels}")

model = load_model("./resnet_model.pth", num_labels, device)
predict(model, train_loader, device, class_labels)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Image 1:
  Predicted Labels: ['Mass', 'Nodule', 'Pleural_Thickening']
  True Labels: ['Mass', 'Nodule', 'Pleural_Thickening']
Image 2:
  Predicted Labels: []
  True Labels: []
Image 3:
  Predicted Labels: ['Fibrosis', 'Pleural_Thickening']
  True Labels: ['Fibrosis', 'Pleural_Thickening']
Image 4:
  Predicted Labels: []
  True Labels: []
Image 5:
  Predicted Labels: ['Atelectasis', 'Infiltration']
  True Labels: ['Atelectasis', 'Infiltration']
